In [0]:
# Defining Schema & Importing Data

from pyspark.sql.types import StructField,StructType,DateType,IntegerType,StringType
schema = StructType([
  StructField("Product_id" , IntegerType() , True),
  StructField("Customer_id" , StringType() , True),
  StructField("Order_date" , DateType() , True),
  StructField("Location" , StringType() , True),
  StructField("Source_Order" , StringType() , True)
  ])

df = spark.read.format('csv') \
  .schema(schema) \
  .load('/FileStore/tables/sales_csv.txt')

df.show(5)

+----------+-----------+----------+--------+------------+
|Product_id|Customer_id|Order_date|Location|Source_Order|
+----------+-----------+----------+--------+------------+
|         1|          A|2023-01-01|   India|      Swiggy|
|         2|          A|2022-01-01|   India|      Swiggy|
|         2|          A|2023-01-07|   India|      Swiggy|
|         3|          A|2023-01-10|   India|  Restaurant|
|         3|          A|2022-01-11|   India|      Swiggy|
+----------+-----------+----------+--------+------------+
only showing top 5 rows



In [0]:
# Add " year , Month , Quarter " Columns

from pyspark.sql.functions import year , month , quarter
df_t1 = df.withColumn('Year' , year(df['Order_date']))
df_t2 = df_t1.withColumn('month' , month(df['Order_date']))
df_t3 = df_t2.withColumn('quarter' , quarter(df['Order_date']))

df_t3.show(4)

+----------+-----------+----------+--------+------------+----+-----+-------+
|Product_id|Customer_id|Order_date|Location|Source_Order|Year|month|quarter|
+----------+-----------+----------+--------+------------+----+-----+-------+
|         1|          A|2023-01-01|   India|      Swiggy|2023|    1|      1|
|         2|          A|2022-01-01|   India|      Swiggy|2022|    1|      1|
|         2|          A|2023-01-07|   India|      Swiggy|2023|    1|      1|
|         3|          A|2023-01-10|   India|  Restaurant|2023|    1|      1|
+----------+-----------+----------+--------+------------+----+-----+-------+
only showing top 4 rows



In [0]:
# Importing Menu Data

schema = StructType([
  StructField("Product_id" , IntegerType() , True),
  StructField("Product_Name" , StringType() , True),
  StructField("Price" , StringType() , True),

  ])

Menu = spark.read.format('csv') \
  .schema(schema) \
  .load('/FileStore/tables/menu_csv.txt')

Menu.show(5)

+----------+------------+-----+
|Product_id|Product_Name|Price|
+----------+------------+-----+
|         1|       PIZZA|  100|
|         2|     Chowmin|  150|
|         3|    sandwich|  120|
|         4|        Dosa|  110|
|         5|     Biryani|   80|
+----------+------------+-----+
only showing top 5 rows



In [0]:
# Merge Sales & Menu

Sales = df_t3
merge = Sales.join(Menu , 'Product_id' , 'inner')
merge.show(4)

+----------+-----------+----------+--------+------------+----+-----+-------+------------+-----+
|Product_id|Customer_id|Order_date|Location|Source_Order|Year|month|quarter|Product_Name|Price|
+----------+-----------+----------+--------+------------+----+-----+-------+------------+-----+
|         1|          A|2023-01-01|   India|      Swiggy|2023|    1|      1|       PIZZA|  100|
|         2|          A|2022-01-01|   India|      Swiggy|2022|    1|      1|     Chowmin|  150|
|         2|          A|2023-01-07|   India|      Swiggy|2023|    1|      1|     Chowmin|  150|
|         3|          A|2023-01-10|   India|  Restaurant|2023|    1|      1|    sandwich|  120|
+----------+-----------+----------+--------+------------+----+-----+-------+------------+-----+
only showing top 4 rows



In [0]:
# Total amount Spent by Customer

from pyspark.sql.functions import sum,desc
total_amt_Customer = merge.groupBy('Customer_id').agg(sum('Price').alias('total')).orderBy(desc('total'))
display(total_amt_Customer)

Customer_id,total
B,4440.0
A,4260.0
C,2400.0
E,2040.0
D,1200.0


Databricks visualization. Run in Databricks to view.

In [0]:
# Total amount Spent by each food Category

total_amt_food = merge.groupBy('Product_Name').agg(sum('Price').alias('Total')).orderBy(desc('total'))
display(total_amt_food)

Product_Name,Total
sandwich,5760.0
Chowmin,3600.0
PIZZA,2100.0
Dosa,1320.0
Pasta,1080.0
Biryani,480.0


Databricks visualization. Run in Databricks to view.

In [0]:
# Total amount of Sales in each Month

total_amt_Month = merge.groupBy('month').agg(sum('Price').alias('Total')).orderBy('month')
display(total_amt_Month)

month,Total
1,2960.0
2,2730.0
3,910.0
5,2960.0
6,2960.0
7,910.0
11,910.0


Databricks visualization. Run in Databricks to view.

In [0]:
# Yearly Sales

total_amt_Year = merge.groupBy('Year').agg(sum('Price').alias('Total')).orderBy('Year')
display(total_amt_Year)

Year,Total
2022,4350.0
2023,9990.0


Databricks visualization. Run in Databricks to view.

In [0]:
# quarter Sales

total_amt_quarter = merge.groupBy('quarter').agg(sum('Price').alias('Total')).orderBy('quarter')
display(total_amt_quarter)

quarter,Total
1,6600.0
2,5920.0
3,910.0
4,910.0


Databricks visualization. Run in Databricks to view.

In [0]:
# How many times each product purchased ? 

from pyspark.sql.functions import count
product_purchased = merge.groupBy('Product_Name').agg(count('*').alias('cnt_orders')).orderBy(desc('cnt_orders'))
display(product_purchased)

Product_Name,cnt_orders
sandwich,48
Chowmin,24
PIZZA,21
Dosa,12
Pasta,6
Biryani,6


Databricks visualization. Run in Databricks to view.

In [0]:
# Frequency of Customer visiting Restaurant

from pyspark.sql.functions import countDistinct
Customer_purchase1 = merge.filter(merge.Source_Order =='Restaurant').groupBy('Customer_id').agg(countDistinct('Order_date').alias('cnt')).orderBy(desc('cnt'))
display(Customer_purchase1)

Customer_id,cnt
B,6
A,6
E,5
C,3
D,1


Databricks visualization. Run in Databricks to view.

In [0]:
# Total Sales By each Country

Country_sales = merge.groupBy('Location').agg(sum('Price').alias('Total')).orderBy(desc('Total'))
display(Country_sales)

Location,Total
UK,7020.0
India,4860.0
USA,2460.0


Databricks visualization. Run in Databricks to view.

In [0]:
# Total sales by Order Source

source_order_sales = merge.groupBy('source_order').agg(sum('Price').alias('Total')).orderBy(desc('Total'))
display(source_order_sales)

source_order,Total
Swiggy,6330.0
zomato,4920.0
Restaurant,3090.0


Databricks visualization. Run in Databricks to view.